In [0]:
%scala
val nbSuffix="_demo1"

nbSuffix: String = _demo1

In [0]:
%sh
cat /dbfs/FileStore/conf/spark_v1_0_offline.conf

[kensu]
;Kensu Ingestion endpoint
kensu_ingestion_url=https://community-api.kensuapp.com
;token for Kensu Ingestion API
kensu_ingestion_token=
;token for Kensu Expectation API (external app personal access token)
kensu_api_token=

; if report_to_file=True then no credentials needed
report_to_file=True
offline_file_name=demo_notebook3.jsonl

project_name=Databricks demo v2
environment=Databricks dev
logical_data_source_naming_strategy=LastFolderAndFile


compute_stats=True
compute_input_stats=True
compute_output_stats=True

; these are set automatically for Databricks notebooks:
; user_name=databricks user
; code_location,process_name=Notebook name

In [0]:
%scala
io.kensu.sparkcollector.environments.DatabricksCollector.track(spark, "/dbfs/FileStore/conf/spark_v1_0_offline.conf", 
                                                               "report_to_file" -> "False"
                                                               // ,
                                                               //"kensu_ingestion_url" -> "https://community-api.kensuapp.com",
                                                               //"kensu_ingestion_token" -> "TODO"
                                                              )

Kensu Spark collector was already initialized earlier on for this JVM, and this subsequent initialization may not take a full effect, e.g. if you intended to change some configuration (if so please restart the JVM).
Spark version: 3.2.1
Collector version: { assembly_assemblyJarName: kensu-spark-collector-1.0.13-SNAPSHOT_spark-3.2.1.jar, version: 1.0.13-SNAPSHOT, scalaVersion: 2.12.17, gitHeadCommit: Some(927af7b39adda26f3686046635121d52ae782bdd), builtAtString: 2022-10-26 11:15:02.659+0300, builtAtMillis: 1666772102659 }

In [0]:
%scala
import org.apache.spark.sql.functions.expr;

def loadStock(year:String, month:String, stock:String):org.apache.spark.sql.DataFrame = {
  spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(s"/FileStore/Financial_Data_Report/$year/$month/$stock.csv")
    .withColumnRenamed("Adj Close", "AdjClose")
    .withColumn("Intraday_Delta", expr("AdjClose - Open"))
}

import org.apache.spark.sql.functions.expr
loadStock: (year: String, month: String, stock: String)org.apache.spark.sql.DataFrame

In [0]:
%scala
val year = "2021"
val month = "dec"

val apple_stock_df = loadStock(year,month, "Apple")
val apptech_stock_df = loadStock(year,month, "AppTech")
val buzzfeed_stock_df = loadStock(year,month, "Buzzfeed")
val eur_usd_df = loadStock(year,month, "Apple")
val imetal_df = loadStock(year,month, "AppTech")
val microsoft_stock_df = loadStock(year,month, "Buzzfeed")

//display(microsoft_stock_df)
val dfSeq = Seq(apple_stock_df, apptech_stock_df, buzzfeed_stock_df,eur_usd_df,imetal_df,microsoft_stock_df)
val monthly_assets_df = dfSeq.reduce(_ union _)

monthly_assets_df.registerTempTable("tmp_monthly_assets")
//monthly_assets_df.show()

command-266451873421898:15: warning: method registerTempTable in class Dataset is deprecated (since 2.0.0): Use createOrReplaceTempView(viewName) instead.
monthly_assets_df.registerTempTable("tmp_monthly_assets")
 ^
year: String = 2021
month: String = dec
apple_stock_df: org.apache.spark.sql.DataFrame = [Date: timestamp, Symbol: string ... 7 more fields]
apptech_stock_df: org.apache.spark.sql.DataFrame = [Date: timestamp, Symbol: string ... 7 more fields]
buzzfeed_stock_df: org.apache.spark.sql.DataFrame = [Date: timestamp, Symbol: string ... 7 more fields]
eur_usd_df: org.apache.spark.sql.DataFrame = [Date: timestamp, Symbol: string ... 7 more fields]
imetal_df: org.apache.spark.sql.DataFrame = [Date: timestamp, Symbol: string ... 7 more fields]
microsoft_stock_df: org.apache.spark.sql.DataFrame = [Date: timestamp, Symbol: string ... 7 more fields]
dfSeq: Seq[org.apache.spark.sql.DataFrame] = List([Date: timestamp, Symbol: string ... 7 more fields], [Date: timestamp, Symbol: string ... 7 more fields], [Date: timestamp, Symbol: string ... 7 more fields], [Date: timestamp, Symbol: string ... 7 more fields], [Date: timestamp, Symbol: string ... 7 more fields], [Date: timestamp, Symbol: string ... 7 more fields])
monthly_assets_df: org.apache.spark.sql.DataFrame = [Date: timestamp, Symbol: string ... 7 more fields]

In [0]:
%sql
DROP TABLE IF EXISTS financial_data_monthly_assets_demo1


In [0]:
%sql
CREATE OR REPLACE TABLE financial_data_monthly_assets_demo1 USING DELTA AS SELECT * FROM tmp_monthly_assets


num_affected_rows,num_inserted_rows


In [0]:
%scala
// Csv read, Databricks Delta write
spark.sql("select * from tmp_monthly_assets").write.mode("overwrite").format("delta").saveAsTable("financial_data_monthly_assets_vidma" + nbSuffix)

In [0]:
%scala
// Databricks Delta read and Delta write
spark.sql("select * from financial_data_monthly_assets_vidma" + nbSuffix).write.mode("overwrite").format("delta").saveAsTable("delta_read_and_delta_write_vidma_" + nbSuffix)